# 04 Automata

DTM의 상태를 $q_0$, $q_1$, $q_2$라고 책에서 표시하지만
Haskell 프로그램에서는 그냥 0,1,2 정수로 구분해서 표시하겠다.
그러면 심볼도 0과 1이라고 하면 헷갈리니까 a와 b로 해서
$\{a^nb^n \mid n>0 \}$ 언어를 인식하는 DTM을 만들어 보자.
그리고 헤더의 위치도 테입의 어느 칸에 위치하는지도 그림이 아니라 정수 인덱스로 나타내겠다.
처음에 0번 인덱스 칸에서 시작하고 왼쪽으로는 음의 정수 오른쪽으로는 양의 정수 인덱스로 나간다.

In [16]:
import Data.Map

empty -- 비어있는 무한히 긴 테입을 나타낸다

tape1 = fromList $ zip [1..] "aabb" -- 테입을 1번 칸에서 시작해서 aabb로 초기화

tape1
elems tape1

fromList []

fromList [(1,'a'),(2,'a'),(3,'b'),(4,'b')]

"aabb"

In [17]:
tape2 = insert 0 'a' tape1
tape2
elems tape2

fromList [(0,'a'),(1,'a'),(2,'a'),(3,'b'),(4,'b')]

"aaabb"

In [18]:
tape3 = insert 5 'b' tape2

tape3
elems tape3

fromList [(0,'a'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')]

"aaabbb"

In [19]:
-- 직전 결과를 it이라고 지칭할 수 있다
fromList $ zip [1..] "aabb"
insert 0 'a' it
insert 5 'b' it
insert 0 'c' it
elems it

fromList [(1,'a'),(2,'a'),(3,'b'),(4,'b')]

fromList [(0,'a'),(1,'a'),(2,'a'),(3,'b'),(4,'b')]

fromList [(0,'a'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')]

fromList [(0,'c'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')]

"caabbb"

이제 준비운동은 다 했으니 책에 나온대로 우리가 튜링머신처럼 한번 따라해 봅시다.

In [20]:
import Data.Map
import qualified Data.Map as Map

data Move = L | N | R deriving (Eq,Ord,Show)

type Tape = Map Int Char

type State = Int
type Loc = Int

type Delta = Map (State, Maybe Char) (State, Maybe Char, Move)

type DTM = ( Int     -- 상태 최대값
           , Delta   -- 전이 규칙
           , Int     -- 최종 상태
           )

type Config = ( Int -- DTM 상태
              , Int -- DTM 헤더 위치
              , Tape
              )

In [21]:
-- a^nb^n 검사하는 튜링머신 정의
anbnDTM = (5, anbnRules, 5)

-- 결정적튜링머신 실행규칙
anbnRules = fromList
    [ ((0,a), (1,c,R))
    , ((1,a), (1,a,R))
    , ((1,b), (2,d,L))
    , ((1,d), (1,d,R))
    , ((2,a), (2,a,L))
    , ((2,c), (3,c,R))
    , ((2,d), (2,d,L))
    , ((3,a), (1,c,R))
    , ((3,d), (4,d,R))
    , ((4,d), (4,d,R))
    , ((4,n), (5,n,N))
    ]

a = Just 'a'
b = Just 'b'
x = Just 'X'
y = Just 'Y'
n = Nothing  -- 공백 기호 책에서는 B

In [22]:
tape0 = fromList $ zip [0..] "aaabbb"
-- 일단 이런 테입에서 튜링머신이 돌아가기 시작
tape0
elems tape0

fromList [(0,'a'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')]

"aaabbb"

In [23]:
Map.lookup (-1) tape0
Map.lookup    0 tape0
Map.lookup    5 tape0
Map.lookup    6 tape0

Nothing

Just 'a'

Just 'b'

Nothing

In [38]:
-- 튜링머신 실행상태
config0 = (0,0,tape0)

move L i = i-1
move N i = i
move R i = i+1

step :: DTM -> (State, Loc, Tape) -> (State, Loc, Tape)
step (_,rules,_) (q, i, tp) = (q', move mv i, tp')
  where
  s = Map.lookup i tp  -- s는 테입의 i번째 인덱스 내용
  Just (q', s', mv) = Map.lookup (q,s) rules
  tp' = case s' of  Nothing -> tp
                    Just ch -> insert i ch tp

run dtm@(_,_,qf) = takeWhile (\(q,_,_) -> q/=qf) . iterate (step dtm)

In [39]:
config0
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it
step anbnDTM it

(0,0,fromList [(0,'a'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')])

(1,1,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')])

(1,2,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')])

(1,3,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')])

(2,2,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'d'),(4,'b'),(5,'b')])

...

(3,3,fromList [(0,'c'),(1,'c'),(2,'c'),(3,'d'),(4,'d'),(5,'d')])

(4,4,fromList [(0,'c'),(1,'c'),(2,'c'),(3,'d'),(4,'d'),(5,'d')])

(4,5,fromList [(0,'c'),(1,'c'),(2,'c'),(3,'d'),(4,'d'),(5,'d')])

(4,6,fromList [(0,'c'),(1,'c'),(2,'c'),(3,'d'),(4,'d'),(5,'d')])

(5,6,fromList [(0,'c'),(1,'c'),(2,'c'),(3,'d'),(4,'d'),(5,'d')])

In [45]:
run anbnDTM config0

mapM_ print it

[(0,0,fromList [(0,'a'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')]),(1,1,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')]),(1,2,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')]),(1,3,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')]),(2,2,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'d'),(4,'b'),(5,'b')]),(2,1,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'d'),(4,'b'),(5,'b')]),(2,0,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'d'),(4,'b'),(5,'b')]),(3,1,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'d'),(4,'b'),(5,'b')]),(1,2,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'b'),(5,'b')]),(1,3,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'b'),(5,'b')]),(1,4,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'b'),(5,'b')]),(2,3,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'d'),(5,'b')]),(2,2,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'d'),(5,'b')]),(2,1,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'d'),(5,'b')]),(3,2,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'d'),(5,'b')]),(1,3,fromList [(0,'c'),(

(0,0,fromList [(0,'a'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')])
(1,1,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')])
(1,2,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')])
(1,3,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'b'),(4,'b'),(5,'b')])
(2,2,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'d'),(4,'b'),(5,'b')])
(2,1,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'d'),(4,'b'),(5,'b')])
(2,0,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'d'),(4,'b'),(5,'b')])
(3,1,fromList [(0,'c'),(1,'a'),(2,'a'),(3,'d'),(4,'b'),(5,'b')])
(1,2,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'b'),(5,'b')])
(1,3,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'b'),(5,'b')])
(1,4,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'b'),(5,'b')])
(2,3,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'d'),(5,'b')])
(2,2,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'d'),(5,'b')])
(2,1,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'d'),(5,'b')])
(3,2,fromList [(0,'c'),(1,'c'),(2,'a'),(3,'d'),(4,'d'),(5,'b')])
(1,3,fromList [(0,'c'),(1